In [24]:
import time 
import logging
import requests
import re
import sys
from bs4 import BeautifulSoup
from queue import Queue
from urllib import parse, request
from urllib.parse import urlparse
import heapq
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.common.by import By

# Public Access Computer Science Course Database

Run the code block below to view an expandable list of all courses in the database. Courses sources from the MIT OpenCourseware webiste and Coursera. 

In [25]:
courses = pd.read_csv("https://raw.githubusercontent.com/Lois-Wong/cs-course-recommender/main/all_courses.csv")
courses = courses.drop(columns=courses.columns[:1])
for index, value in enumerate(courses["Description"]):
    # Example editing: appending " - edited" to each value
    courses.at[index, 'Description'] = value.replace("\n", "")


# Increment all row numbers by 1
courses.index += 1

import ipywidgets as widgets
from IPython.display import display, HTML

# Set the display options for pandas
pd.set_option('display.max_colwidth', None)  # Display full text in each cell

# Define initial display parameters
start_index = 10
rows_per_page = 10

# Define the function to display more rows
def show_more_rows(button):
    global start_index
    with output:
        end_index = start_index + rows_per_page
        if end_index >= len(courses):
            end_index = len(courses)
            button.disabled = True
        display(courses.iloc[start_index:end_index])
        start_index = end_index

# Display the second set of ten rows (rows 10-19)
display(courses.iloc[0:10])

# Create a button widget to show more rows
button = widgets.Button(description="Show More Rows")
button.on_click(show_more_rows)

# Create an output widget to display the DataFrame
output = widgets.Output(layout=widgets.Layout(width='100%', height='100%', overflow='scroll'))

# Display the button widget
display(output)
display(button)


,Title,Description,Link,Source
1,Mathematics for Computer Science,"This course covers elementary discrete mathematics for computer science and engineering. It emphasizes mathematical definitions and proofs as well as applicable methods. Topics include formal logic notation, proof methods; induction, well-ordering; sets, relations; elementary graph theory; integer congruences; asymptotic notation and growth of functions; permutations and combinations, counting principles; discrete probability. Further selected topics may also be covered, such as recursive definition and structural induction; state machines and invariants; recurrences; generating functions.",https://ocw.mit.edu/courses/6-042j-mathematics-for-computer-science-fall-2010/,MIT OpenCourseware
2,Mathematics for Computer Science,"This is an introductory course in Discrete Mathematics oriented toward Computer Science and Engineering. The course divides roughly into thirds:Fundamental Concepts of Mathematics: Definitions, Proofs, Sets, Functions, RelationsDiscrete Structures: Modular Arithmetic, Graphs, State Machines, CountingDiscrete Probability TheoryA version of this course from a previous term was also taught as part of the Singapore-MIT Alliance (SMA) programme as course number SMA 5512 (Mathematics for Computer Science).",https://ocw.mit.edu/courses/6-042j-mathematics-for-computer-science-fall-2005/,MIT OpenCourseware
3,Mathematics for Computer Science,"This subject offers an interactive introduction to discrete mathematics oriented toward computer science and engineering. The subject coverage divides roughly into thirds:Fundamental concepts of mathematics: Definitions, proofs, sets, functions, relations.Discrete structures: graphs, state machines, modular arithmetic, counting.Discrete probability theory.On completion of 6.042J, students will be able to explain and apply the basic methods of discrete (noncontinuous) mathematics in computer science. They will be able to use these methods in subsequent courses in the design and analysis of algorithms, computability theory, software engineering, and computer systems.This course is part of the Open Learning Library, which is free to use. You have the option to sign up and enroll in the course if you want to track your progress, or you can view and use all the materials without enrolling.",https://ocw.mit.edu/courses/6-042j-mathematics-for-computer-science-spring-2015/,MIT OpenCourseware
4,Computational Cognitive Science,"This course is an introduction to computational theories of human cognition. Drawing on formal models from classic and contemporary artificial intelligence, students will explore fundamental issues in human knowledge representation, inductive learning and reasoning. What are the forms that our knowledge of the world takes? What are the inductive principles that allow us to acquire new knowledge from the interaction of prior knowledge with observed data? What kinds of data must be available to human learners, and what kinds of innate knowledge (if any) must they have?",https://ocw.mit.edu/courses/9-66j-computational-cognitive-science-fall-2004/,MIT OpenCourseware
5,Great Ideas in Theoretical Computer Science,"This course provides a challenging introduction to some of the central ideas of theoretical computer science. It attempts to present a vision of “computer science beyond computers”: that is, CS as a set of mathematical tools for understanding complex systems such as universes and minds. Beginning in antiquity—with Euclid’s algorithm and other ancient examples of computational thinking—the course will progress rapidly through propositional logic, Turing machines and computability, finite automata, Gödel’s theorems, efficient algorithms and reducibility, NP-completeness, the P versus NP problem, decision trees and other concrete computational models, the power of randomness, cryptography and one-way functions, computational theories of learning, interactive proofs, and quantum computing and the phys

Output(layout=Layout(height='100%', overflow='scroll', width='100%'))

Button(description='Show More Rows', style=ButtonStyle())

# Search the database

Run the code block below and enter a search token. The title and description of each course will be searched for the token you entered. All matches will be outputted as an expandable list. Run this block again to do another search. 

In [26]:
import pandas as pd


# Function to filter rows based on input token in title or description
def filter_rows(input_token):
    filtered_rows = pd.DataFrame(columns=courses.columns)
    for index, row in courses.iterrows():
        title = str(row['Title']).lower()
        description = str(row['Description']).lower()
        # Check if the input token is present in title or description
        if input_token.lower() in title or input_token.lower() in description:
            filtered_rows.loc[len(filtered_rows)] = row
    return filtered_rows


# User input for token
input_token = input("Enter a token: ")

# Filter rows based on input token
filtered_courses = filter_rows(input_token)

filtered_courses.index += 1

if len(filtered_courses) > 0:
    pd.set_option('display.max_colwidth', None)  # Display full text in each cell

    # Define initial display parameters
    start_index = 10
    rows_per_page = 10

    # Define the function to display more rows
    def show_more_rows(button):
        global start_index
        with output:
            end_index = start_index + rows_per_page
            if end_index >= len(filtered_courses):
                end_index = len(filtered_courses)
                button.disabled = True
            display(filtered_courses.iloc[start_index:end_index])
            start_index = end_index

# Display the second set of ten rows (rows 10-19)
    display(filtered_courses.iloc[0:10])

# Create a button widget to show more rows
    button = widgets.Button(description="Show More Rows")
    button.on_click(show_more_rows)

# Create an output widget to display the DataFrame
    output = widgets.Output(layout=widgets.Layout(width='100%', height='100%', overflow='scroll'))

# Display the button widget
    display(output)
    display(button)
else:
    print("No matches")

Enter a token: C++


,Title,Description,Link,Source
1,High Speed Communication Circuits and Systems,"6.976 covers circuit and system level design issues of high speed communication systems, with primary focus being placed on wireless and broadband data link applications. Specific circuit topics include transmission lines, high speed and low noise amplifiers, VCO’s, and high speed digital circuits. Specific system topics include frequency synthesizers, clock and data recovery circuits, and GMSK transceivers. In addition to learning analysis skills for the above items, students will gain a significant amount of experience in simulating circuits in SPICE and systems in CppSim (a custom C++ simulator).",https://ocw.mit.edu/courses/6-976-high-speed-communication-circuits-and-systems-spring-2003/,MIT OpenCourseware
2,Introduction to C and C++,"This course provides a fast-paced introduction to the C and C++ programming languages. You will learn the required background knowledge, including memory management, pointers, preprocessor macros, object-oriented programming, and how to find bugs when you inevitably use any of those incorrectly. There will be daily assignments and a small-scale individual project.This course is offered during the Independent Activities Period (IAP), which is a special 4-week term at MIT that runs from the first week of January until the end of the month.",https://ocw.mit.edu/courses/6-s096-introduction-to-c-and-c-january-iap-2013/,MIT OpenCourseware
3,Effective Programming in C and C++,"This course is a fast-paced introduction to the C and C++ programming languages, with an emphasis on good programming practices and how to be an effective programmer in these languages. Topics include object-oriented programming, memory management, advantages of C and C++, optimization, and others. Students are given weekly coding assignments and a final project to hone their skills. Recommended for programmers with some background and experience in other languages.This course is offered during the Independent Activities Period (IAP), which is a special 4-week term at MIT that runs from the first week of January until the end of the month.",https://ocw.mit.edu/courses/6-s096-effective-programming-in-c-and-c-january-iap-2014/,MIT OpenCourseware
4,Introduction to C++,"This is a fast-paced introductory course to the C++ programming language. It is intended for those with little programming background, though prior programming experience will make it easier, and those with previous experience will still learn C++-specific constructs and concepts.This course is offered during the Independent Activities Period (IAP), which is a special 4-week term at MIT that runs from the first week of January until the end of the month.",https://ocw.mit.edu/courses/6-096-introduction-to-c-january-iap-2011/,MIT OpenCourseware
5,Introduction to C Memory Management and C++ Object-Oriented Programming,"Ever hang your head in shame after your Python program wasn’t as fast as your friend’s C program? Ever wish you could use objects without having to use Java? Join us for this fun introduction to C and C++! We will take you through a tour that will start with writing simple C programs, go deep into the caves of C memory manipulation, resurface with an introduction to using C++ classes, dive deeper into advanced C++ class use and the C++ Standard Template Libraries. We’ll wrap up by teaching you some tricks of the trade that you may need for tech interviews.We see this as a “C/C++ empowerment” course: we want you to come away understandingwhy you would want to use C over another language (control over memory, probably for performance reasons),why you would want to use C++ rather than C (objects), andhow to be useful in C and C++.This course is offered during the Independent Activities Period (IAP), which is a special 4-week term at MIT that runs from the first week of January until the end of the month.",https://ocw.mit.edu/courses/6-088-introduction-to-c-memory-management-and

Output(layout=Layout(height='100%', overflow='scroll', width='100%'))

Button(description='Show More Rows', style=ButtonStyle())

# Finding courses that suit your goals

Run the code block below and enter a query that describes your goals for your CS education. After you enter your query, a sorted list of the courses that might best suit your goals will appear. Run the code block again to enter a different query.

In [27]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained sentence transformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# User input
user_goals = input("What are your goals for an online CS course? ")
print("Loading...")

# Course descriptions
course_descriptions = [...]  # Your list of course descriptions

# Encode the user's goals
user_goals_embedding = model.encode(user_goals, convert_to_tensor=True)

# Calculate similarity scores for each course description
similarity_scores = []
for entry in courses.itertuples():
    # Encode the course description
    course_description_embedding = model.encode(entry.Description, convert_to_tensor=True)
    
    # Calculate the similarity between the embeddings
    cosine_score = util.pytorch_cos_sim(user_goals_embedding, course_description_embedding).item()
    
    # Store the similarity score along with the corresponding course description
    similarity_scores.append((cosine_score, entry.Title, entry.Description, entry.Link, entry.Source))

# Sort the courses based on similarity scores in descending order
similarity_scores.sort(key=lambda x: x[0], reverse=True)

# Output the top ten most similar courses in a dataframe
df = pd.DataFrame(similarity_scores, columns=['Similarity Score', 'Title', 'Description', 'Link', 'Source'])

import ipywidgets as widgets
from IPython.display import display, HTML

# Set the display options for pandas
pd.set_option('display.max_colwidth', None)  # Display full text in each cell

# Define initial display parameters
start_index = 10
rows_per_page = 10

# Define the function to display more rows
def show_more_rows(button):
    global start_index
    with output:
        end_index = start_index + rows_per_page
        if end_index >= len(df):
            end_index = len(df)
            button.disabled = True
        display(df.iloc[start_index:end_index])
        start_index = end_index

# Display the second set of ten rows (rows 10-19)
display(df.iloc[0:10])

# Create a button widget to show more rows
button = widgets.Button(description="Show More Rows")
button.on_click(show_more_rows)

# Create an output widget to display the DataFrame
output = widgets.Output(layout=widgets.Layout(width='100%', height='100%', overflow='scroll'))

# Display the button widget
display(output)
display(button)


/Users/amandaferber/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


What are your goals for an online CS course? Learning how to code.
Loading...


,Similarity Score,Title,Description,Link,Source
0,0.661412,Programming with Scratch,"Learning coding is not only about understanding the programming language being used, but also developing important computational thinking skills, which are useful for problem solving across many disciplinary areas. In this course, students will learn basic programming skills by creating interactive storybooks, animations, and games with Scratch, which is a block-based visual programming language for anyone new to coding.",https://www.coursera.org/learn/programming-with-scratch,Coursera
1,0.591349,"Writing, Running, and Fixing Code in C","Building on the course Programming Fundamentals, this course will teach you how to write code by first planning what your program should do—an important approach for novice and professional programmers. You will learn how to compile and run your program, and then how to test and debug it. This course builds on the Seven Steps you have already learned and provides a framework for systematically testing for problems and fixing them, so you can find and fix problems efficiently.",https://www.coursera.org/learn/writing-running-fixing-code,Coursera
2,0.565770,Python Basics: Automation and Bots,"Understanding the flow of running code is a major part of learning to think in code and of coding itself. In this course we will study the flow of code through several demonstrations and walkthroughs. We'll experience turning logic into useful work by running Python that automatically reads all of Shakespeare, and by setting Python up to give you a call on the phone. In technical terms, this course will demonstrate Python loops, list comprehensions, and conditional statements, while at a higher level we'll discuss code style and good practices for code.",https://www.coursera.org/learn/python-basics-automation-and-bots,Coursera
3,0.559011,Java Programming: Solving Problems with Software,"Learn to code in Java and improve your programming and problem-solving skills. You will learn to design algorithms as well as develop and debug programs. Using custom open-source classes, you will write programs that access and transform images, websites, and other types of data. At the end of the course you will build a program that determines the popularity of different baby names in the US over time by analyzing comma separated value (CSV) files. After completing this course you will be able to:1. Edit, compile, and run a Java program;2. Use conditionals and loops in a Java program;3. Use Java API documentation in writing programs. 4. Debug a Java program using the scientific method;5. Write a Java method to solve a specific problem;6. Develop a set of test cases as part of developing a program;7. Create a class with multiple methods that work together to solve a problem; and8. Use divide-and-conquer design techniques for a program that uses multiple methods.",https://www.coursera.org/learn/java-programming,Coursera
4,0.546148,C++ Basics: Selection and Iteration,"Code and run your first C++ program in minutes without installing anything! This course is designed for learners with no coding experience, providing a solid foundation of not just C++, but core Computer Science topics that can be transferred to other languages. The modules in this course cover printing, operators, iteration (i.e., loops), and selection (i.e., conditionals).To allow for a truly hands-on, self-paced learning experience, this course is video-free. Assignments contain short explanations with images and runnable code examples with suggested edits to explore code examples further, building a deeper understanding by doing. You'll benefit from instant feedback from a variety of assessment items along the way, gently progressing from quick understanding checks (multiple choice, fill in the blank, and un-scrambling code blocks) to small, approachable coding exercises that take minutes instead of hours.",https://www.coursera.org/learn/codio-cpp-basics,Coursera
5,0.541757,In

Output(layout=Layout(height='100%', overflow='scroll', width='100%'))

Button(description='Show More Rows', style=ButtonStyle())